<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 4**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

**Requisitos mínimos:**

* **Funciones** que generen trayectorias tipo Brownian Motion (**BM**), Correlated Random Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el **numero de pasos**, la **velocidad** y **posición inicial**.
  * Además la funciones para **CRW** y **LF** deberán tomar como parámetro el **coeficiente** para la distribución Cauchy.
  * Por último, la función para **LF** también deberá también aceptar como parámetro el **exponente Lévy** (alpha).
* **Funciones** para calcular las métricas **Path length** and **Mean Squared Displacement**.

**El dashboard degerá contener al menos los siguientes elementos:**

* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcular de la trayectoria bajo análisis (path length o mean squared displacement).
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a generar y análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante (segun sea el caso) para ajustar los parámetros de las trayectorias. Estos widgets deberán mostrarse de manera dinámica, es decir, solo deberan mostrarse los parametros relevantes para cada tipo de trayectoria.

## ***Módulos***

In [17]:
import math
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance
pn.extension('plotly')

## **Clases**

In [18]:
# Class Vec2d
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## **Funciones**

In [19]:
# Movimiento Browniano Bidimencional (2D)
def bm_2d(pasos=1000, velocity=6,posIniX=0,posIniY=0):
  velocity = Vec2d(velocity,0)
  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos', 'msd', 'pl'])
  temp_df = pd.DataFrame([{'x_pos':posIniX, 'y_pos':posIniY, 'z_pos':0, 'msd':0, 'pl':0}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  # Generamos las trayectorias aleatorias
  for i in range(0,pasos-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi) # Ángulo de giro aleatorio entre -pi y pi
    velocity = velocity.rotated(turn_angle)
    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y, 'z_pos':i+1, 'msd':0, 'pl':0}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

In [4]:
## Caminata aleatoria c ##

def CRW_2d(pasos=1000, velocity=6, posIniX=0, posIniY=0, factorCauchy=0.4,):
  #se usa clase
  velocity = Vec2d(velocity,0)

  #estructura de dataframe pd
  CRW_2d_df = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos', 'msd', 'pl'])
  temp_df = pd.DataFrame([{'x_pos':posIniX, 'y_pos':posIniY, 'z_pos':0,'msd':0, 'pl':0}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
  ListaPDTrayectorias = []
  distribucionDireccion = wrapcauchy(factorCauchy,0,scale=1)
  listaDirecciones = distribucionDireccion.rvs(size=pasos)
  for i in range(0, pasos -1):
    turn_angle = listaDirecciones[i]
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i]+velocity.x,'y_pos':CRW_2d_df.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)
  return CRW_2d_df

In [5]:
## Levy Flight ##

def LevyFlyPd(pasos,speed, miu, levyExponent, levyBeta, factorCauchy, posIniX=0, posIniY=0):
  s_pos = [0,0]
  velocity = Vec2d(speed,0)

  # Instancia de objeto Wrapcauchy
  distribucionDireccion = wrapcauchy(0.4,0,scale=1)

  # Cálculo de giros para trayectorias
  listaDirecciones = distribucionDireccion.rvs(size=pasos)

  # Instancia de objeto Levy_stable
  distribucionPasos = levy_stable(alpha=levyExponent, beta=levyBeta, loc=miu)

  # Cálculo de pasos para la trayectoria
  listaCantidadPasos = distribucionPasos.rvs(size=pasos)

  # Creamos el DataFrame con Pandas
  pdLevy = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos', 'msd', 'pl'])
  temp3d = pd.DataFrame([{'x_pos':posIniX,'y_pos':posIniY,'z_pos':0,'msd':0,'pl':0}])
  pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
  PasosPendientes=0
  IndicePasos = 0
  IndiceCambiosDireccion = 0

  for i in range(0,pasos-1):
    if PasosPendientes < 1:
      velocity=velocity.rotated(listaDirecciones[IndiceCambiosDireccion])
      IndiceCambiosDireccion=IndiceCambiosDireccion + 1
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy, temp3d], ignore_index=True)
      PasosPendientes=int(listaCantidadPasos[IndicePasos])
      IndicePasos=IndicePasos+1
    else:
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=PasosPendientes-1

    return pdLevy

In [7]:
## cálculo de distancia ##
def distanciaXYmean(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)

  return distancia

def distanciaXY(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)
  return distancia

## Msd ##
def trajectoryMean(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  TrajectoryLenghtAC = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'s_pos':0,'msd':0, 'pl':0}])
  TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  for i in range(1,pasos-1):
    contadorProm=0
    distanciaRetorno = 0.0
    for ii in range(0,pasos-i):
      contadorProm=contadorProm+1
      distanciaRetorno=distanciaRetorno + (distanciaXYmean(trayectoria.x_pos[ii],trayectoria.y_pos[ii],trayectoria.x_pos[ii+i],trayectoria.y_pos[ii+i]))**2
    distanciaRetorno=(distanciaRetorno / contadorProm)
    temp_df = pd.DataFrame([{'x_pos':trayectoria.x_pos[i-1],'y_pos':trayectoria.y_pos[i-1],'z_pos':trayectoria.z_pos[i-1],'msd':distanciaRetorno,'pl':trayectoria.pl[i-1]}])
    TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  return TrajectoryLenghtAC

## Path Lenght
def trajectoryLenght(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  TrayectoryLenghtAC = pd.DataFrame(columns=['x_pos','y_pos','z_pos',',msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'z_pos':0,'msd':0,'pl':0}])
  TrajectoryLenghtAC = pd.concat([TrayectoryLenghtAC, temp_df], ignore_index=True)
  for i in range(1,pasos):
    distanciaRetorno=distanciaRetorno + distanciaXY(trayectoria.x_pos[i-1],trayectoria.y_pos[i-1],trayectoria.x_pos[i],trayectoria.y_pos[i])
    temp_df = pd.DataFrame([{'x_pos':trayectoria.x_pos[i],'y_pos':trayectoria.y_pos[i],'z_pos':i, 'msd':trayectoria.msd[i],'pl':distanciaRetorno}])
    TrayectoriLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  return TrajectoryLenghtAC

## Trayectoria 3D ##
def GraficaTrayectoria3D(trayectoria,comentario='BM Walk MSD'):
  fig3D = go.Figure(data =[go.Scatter3d(x= trayectoria.x_pos,
                                    y= trayectoria.y_pos,
                                    z = trayectoria.z_pos,
                                    marker=dict(size=3),
                                    line = dict(width=3),
                                    mode = 'lines',
                                    name = comentario,
                                    showlegend = True)])
  return fig3D

In [15]:
# Función dinámica para generar métricas con base a parámetros
def GraficaMetrica(trayectoria, metrica=1, comentario='BM Walk MSD'):
  GraficaMetrica = go.Figure()
  if metrica == 1:
    GraficaMetrica.add_trace(go.Scatter(
      x = trayectoria.z_pos,
      y = trayectoria.msd,
      marker = dict(size = 3),
      line = dict(width = 1),
      mode = 'lines',
      name = comentario,
      showlegend = True
    ))
  else:
    GraficaMetrica.add_trace(go.Scatter(
        x = trayectoria.z_pos,
        y = trayectoria.pl,
        marker = dict(size = 3),
        line = dict(width = 1),
        mode = 'lines',
        name = comentario,
        showlegend = True
    ))
  return GraficaMetrica

In [16]:
# Pruebas de gráficas sin interfaz
bmt = bm_2d(1000,6,2,2)
crwt = CRW_2d(1000,6,9, 5,0.4)
lft = LevyFlyPd(1000,6,3,0.9,1,0.4,5,5)

GraficaMetrica(trajectoryMean(bmt),1,'MSD BM').show()
GraficaMetrica(trajectoryLenght(bmt),2,'PL BM').show()
GraficaTrayectoria3D(bmt,'BM walk').show()

GraficaTrayectoria3D(bm_2d(1000,6,2,2),'BM walk').show()
GraficaTrayectoria3D(CRW_2d(1000,6,9,5,0.4),'CRW Walk').show()
GraficaTrayectoria3D(LevyFlyPd(1000,6,3,0.9,0,0.4,5,5),'Levy Flight').show()

GraficaMetrica(trajectoryMean(crwt),1,'MSD CRW').show()
GraficaMetrica(trajectoryLenght(crwt),2,'PL CRW').show()
GraficaTrayectoria3D(crwt,'BM walk').show()

## **Dashboard**

In [ ]:
TipoTrayectoria = pnw.Select(name='Tipo de trayectoria a calcular', width=150, value='BM', options=['BM', 'CRW', 'Levy'])
Numero_pasos = pnw.IntSlider(name='Numero de pasos', width=150, value=500, step=50, start=0, end=2000)
posIX = pnw.IntInput(name='Posicion X inicial', width=150,value=0, step=1, start=0, end=2000)
posIY = pnw.IntInput(name='Posicion Y inicial', width=150,value=0, step=1, start=0, end=2000)
Velocidad = pnw.IntSlider(name='Velocidad',width=150,value=5,step=1, start=1, end=20)
Metrica = pnw.Select(name='Tipo de Metrica', width=150, value= 'PL', options=['MSD', 'PL'])
fCauchy = pnw.FloatSlider(name='Fac Cauchy', width=150, value=0.4, step=0.1, start = 0.0, end=2.0)
Levymiu = pnw.IntSlider(name='Levy Beta', width=150, value=3, step=1, start=0, end=10)
LevyBeta = pnw.FloatSlider(name='Levy Beta', width=150, value=1.0, step=0.1, start=0, end=0.2)
LevyExponent=pnw.FloatSlider(name='LevyExponent', width=150, value=0.9, step=0.1, start=0, end=2.0)
fCauchyLevy=pnw.FloatSlider(name='Factor Cauchy Levy', width=150, value=0.4, step=0.1, start=0, end=2.0)
bmt=bm_2d(500,5,0,0)
crwt=CRW_2d(500,5,0,0,0.4)
lft=LevyFlyPd(500,5,3,0.9,1,0.4,0,0)
GraficaMet=GraficaMetrica(trajectoryLenght(bmt),2,'PL BM')
GraficaTray=GraficaTrayectoria3D(bmt,'BM walk')
LocalTipoTray=''
@pn.depends(TipoTrayectoria, Numero_pasos,posIX,posIY,Velocidad,fCauchy,Levymiu,LevyBeta,LevyExponent,fCauchyLevy)
def figuratray3d(TipoTrayectoria, Numero_pasos,posIX,posIY,Velocidad,fCauchy,Levymiu,LevyBeta,LevyExponent,fCauchyLevy):
  if TipoTrayectoria=='BM':
    LocalTipoTray=TipoTrayectoria
    print('EntroBM')
    bmt=bm_2d(Numero_pasos,Velocidad,posIX,posIY)
    GraficaTray=GraficaTrayectoria3D(bmt,'BM walk')
    return GraficaTray
  elif TipoTrayectoria =='CRW':
    LocalTipoTray=TipoTrayectoria
    print('EntroCRW')
    crwt=CRW_2d(Numero_pasos,Velocidad,posIX,posIY,fCauchy)
    GraficaTray=GraficaTrayectoria3D(crwt, 'CRW walk')
    return GraficaTray
  elif TipoTrayectoria =='Levy':
    LocalTipoTray=TipoTrayectoria
    print('EntroLevy')
    lft=LevyFlyPd(Numero_pasos, Velocidad, Levymiu, LevyExponent, LevyBeta, fCauchyLevy, posIX, posIY)
    GraficaTray=GraficaTrayectoria3D(lft,'levy walk')
    return GraficaTray

@pn.depends(TipoTrayectoria,Metrica)
def figuraMetrica(TipoTrayectoria,Metrica):
  if TipoTrayectoria=='BM':
    if Metrica == 'PL':
      grafica = GraficaMetrica(trajectoryLenght(bmt),2,'PL BM')
    else:
      grafica = GraficaMetrica(trajectoryMean(bmt),1,'MSD BM')
    return grafica
  elif TipoTrayectoria == 'Levy':
    if Metrica == 'PL':
      grafica = GraficaMetrica(trajectoryLenght(lft),2,'PL Levy')
    else:
      grafica = GraficaMetrica(trajectoryMean(lft),1,'MSD Levy')
    return grafica

## Trayectoria Tipo --
pn.Row(pn.Column(TipoTrayectoria,Numero_pasos,posIX,posIY,Velocidad,Metrica,fCauchy,Levymiu,LevyBeta,LevyExponent,fCauchyLevy),figuratray3d,figuraMetrica)


In [ ]:
TipoTrayectoria = pnw.Select(name='Tipo de trayectoria a calcular', width=150, value='BM', options=['BM','CRW','Levy'],visible=False)
TipoTrayectoriaBM = pnw.Button(name='BM', button_type='success', disabled=True)
TipoTrayectoriaCRW = pnw.Button(name='CRW', button_type='success')
TipoTrayectoriaLevy = pnw.Button(name='Levy',button_type='success')

Numero_pasos = pnw.IntSlider(name='Numero de pasos', width=150, value=500, step=50, start=0, end=2000)
posIX = pnw.IntInput(name='Posicion X inicial', width=150, value=0, step=1, start=0, end=2000)
PosIY = pnw.IntInput(name='Posicion Y inicial', width=150, value=0, step=1, start=0, end=2000)
Velocidad=pnw.IntSlider(name='Velocidad',width=150, value=5, step=1, start=1, end=20)
Metrica = pnw.Select(name='Tipo de Metrica', width=150, value='PL',options=['MSD','PL'])
fCauchy=pnw.FloatSlider(name='Factor Cauchy', width=150, value=0.4, step=0.1, start=0.0, end=1.0, visible=False)
Levymiu=pnw.IntSlider(name='Levy Beta',width=150, value=3, step=1, start=0, end=10, visible=False)
LevyBeta=pnw.FloatSlider(name='LevyBeta',width=150, value=1.0, step=0.1, start=0, end=2.0, visible=False)
LevyExponent=pnw.FloatSlider(name='Levy Exponent', width=150, value=0.9, step=0.1, start=0, end=2.0, visible=False)
fCauchyLevy=pnw.FloatSlider(name='Factor Cauchy Levy', width=150, value=0.4, step=0.1, start=0, end=2.0, visible=False)

@pn.depends(TipoTrayectoria, Numero_pasos,posIX,posIY,Velocidad,fCauchy,Levymiu,LevyBeta,LevyExponent,fCauchyLevy)
def figuratray3d(TipoTrayectoria, Numero_pasos, posIX, posIY, Velocidad, fCauchy, Levymiu,LevyBeta, LevyExponent, fCauchyLevy):
  if TipoTrayectoria=='BM':
    LocalTipoTray=TipoTrayectoria
    global bmt
    bmt=bm_2d(Numero_pasos,Velocidad,posIX,posIY)
    GraficaTray=GraficaTrayectoria3D(bmt,'BM Walk')
    return GraficaTray
  elif TipoTrayectoria == 'CRW':
    LocalTipoTray=TipoTrayectoria
    global crwt
    crwt=CRW_2d(Numero_pasos,Velocidad,posIX,posIY,fCauchy)
    GraficaTray=GraficaTrayectoria3D(crwt,'CRW walk')
    return GraficaTray
  elif TipoTrayectoria == 'Levy':
    LocalTipoTray=TipoTrayectoria
    global lft
    lft=LevyFlyPd(Numero_pasos,Velocidad,Levymiu, LevyExponent,LevyBeta,fCauchyLevy,posIX,posIY)
    GraficaTray=GraficaTrayectoria3D(lft,'levy walk')
    return GraficaTray

@pn.depends(TipoTrayectoria, Metrica, Numero_pasos, posIX, posIY, Velocidad, fCauchy, Levymiu, LevyBeta, LevyExponent,fCauchyLevy)
def figuraMetrica(TipoTrayectoria,Metrica, Numero_pasos, posIX, posIY, Velocidad, fCauchy, Levymiu, LevyBeta, LevyExponent, fCauchyLevy):
  if TipoTrayectoria=='BM':
    if Metrica == 'PL':
      grafica = GraficaMetrica(trajectoryLenght(bmt),2,'PL BM')
    else:
      grafica = GraficaMetrica(trajectoryMean(bmt),1,'MSD BM')
    return grafica
  elif TipoTrayectoria == 'CRW':
    if Metrica == 'PL':
      grafica = GraficaMetrica(trajectoryLenght(crwt),2,'PL CRW')
    else:
      grafica = GraficaMetrica(trajectoryMean(crwt),1,'MSD CRW')
    return grafica
  elif TipoTrayectoria == 'Levy':
    if Metrica == 'PL':
      grafica = GraficaMetrica(trajectoryLenght(lft),2,'PL Levy')
    else:
      grafica = GraficaMetrica(trajectoryMean(lft),1,'MSD Levy')
    return grafica

#trayectoria Tipos

def toggle_visibilityBM(event):
  if TipoTrayectoriaBM.disaled==False:
    TipoTrayectoriaBM.disables = True
    TipoTrayectoriaCRW.disabled = False
    TipoTrayectoriaLevy.disables=False
    TipoTrayectoria.value='BM'
    fCauchy.visible=False
    Levymiu.visible=False
    LevyBeta.visible=False
    LevyExponent.visible=False
    fCauchyLevy.visible=False

TipoTrayectoriaBM.on_click(toggle_visibilityBM)

def toggle_visibilityCRW(event):
  TipoTrayectoriaCRW.disabled = False
  TipoTrayectoriaBM.disabled = False
  TipoTrayectoriaCRW.disabled = True
  TipoTrayectoriaLevy.disabled=False
  TipoTrayectoria.value='CRW'
  fCauchy.visible=True
  Levymiu.visible=False
  LevyBeta.visible=False
  LevyExponent.visible=False
  fCauchyLevy.visible=False

TipoTrayectoriaCRW.on_click(toggle_visibilityCRW)

def toggle_visibilityLevy(event):
  if TipoTrayectoriaLevy.disables==False:
    TipoTrayectoriaBM.disabld= False
    TipoTrayectoriaCRW.disabled = False
    TipoTrayectoriaLevy.disabled = True
    TipoTrayectoria.value='Levy'
    fCauchy.visible=False
    Levymiu.visible=True
    LevyBeta.visible=True
    LevyExponent.visible=True
    fCauchyLevy.visible=True

TipoTrayectoriaLevy.on_click(toggle_visibilityLevy)

pn.Row(pn.Column(pn.Row(TipoTrayectoriaBM,TipoTrayectoriaCRW,TipoTrayectoriaLevy), TipoTrayectoria, Numero_pasos, posIX, posIY, Velocidad, Metrica, fCauchy, Levymiu, LevyBeta, LevyExponent, fCauchyLevy),figuratray3d,figuraMetrica)